In [6]:
from ctypes import *

In [7]:
lib = CDLL("./tANS_c.so")

lib.connect()

-1

In [ ]:
lib.

In [8]:
# Define the Encoder structure in Python
# class EncodeTable(ctypes.Structure):
#     _fields_ = [
#         ("L", ctypes.c_int),
#         ("s_list", ctypes.POINTER(ctypes.c_uint8)),
#         ("L_s", ctypes.POINTER(ctypes.c_uint8)),
#         ("table", ctypes.POINTER(ctypes.c_int)),
#         ("n_sym", ctypes.c_uint8),
#         ("k", ctypes.POINTER(ctypes.c_uint8)),
#         ("nb", ctypes.POINTER(ctypes.c_int)),
#         ("start", ctypes.POINTER(ctypes.c_int))
#     ]

# class Encoder(ctypes.Structure):
#     _fields_ = [
#         ("state", ctypes.c_int),
#         ("bitstream", ctypes.POINTER(ctypes.c_uint8)),
#         ("bitstream_capacity", ctypes.c_long),
#         ("l_bitstream", ctypes.c_long),
#         ("msg", ctypes.POINTER(ctypes.c_uint8)),
#         ("l_msg", ctypes.c_int),
#         ("ind_msg", ctypes.c_int)
#     ]

# Define the coder structure in Python
class Coder(Structure):
    _fields_ = [
        ("L", c_int),
        ("s_list", POINTER(c_uint8)),
        ("L_s", POINTER(c_uint8)),
        ("n_sym", c_uint8),
        # Define other fields as necessary
    ]


# Define the function prototype for encodeDecodePython
lib.encodeDecodePython.argtypes = [c_int, POINTER(c_uint8), POINTER(c_uint8), c_uint8, POINTER(c_uint8), c_int]
lib.encodeDecodePython.restype = c_int

# Define the function prototype for encodeDecode
lib.encodeDecode.argtypes = [POINTER(Coder), POINTER(c_uint8), c_int]
lib.encodeDecode.restype = c_int

# Define the function prototype for initCoder
lib.initCoder.argtypes = [c_int, POINTER(c_uint8), POINTER(c_uint8), c_uint8]
lib.initCoder.restype = POINTER(Coder)

In [9]:
# Define the Python wrapper function
def encode_decode_python(L, s_list, L_s, n_sym, msg):
    s_list_array = (c_uint8 * len(s_list))(*s_list)
    L_s_array = (c_uint8 * len(L_s))(*L_s)
    msg_array = (c_uint8 * len(msg))(*msg)
    res = lib.encodeDecodePython(L, s_list_array, L_s_array, n_sym, msg_array, len(msg))
    return res

# Wrapper function for initCoder
def init_coder(L, s_list, L_s, n_sym):
    s_list_array = (c_uint8 * len(s_list))(*s_list)
    L_s_array = (c_uint8 * len(L_s))(*L_s)
    coder_ptr = lib.initCoder(L, s_list_array, L_s_array, n_sym)
    return coder_ptr

# Wrapper function for encodeDecode
def encode_decode(coder_ptr, msg):
    msg_array = (c_uint8 * len(msg))(*msg)
    res = lib.encodeDecode(coder_ptr, msg_array, len(msg))
    return res

In [10]:
L = 256 * 2
s_list = [i for i in range(16)]
L_s = [19, 4, 8, 33, 8, 18, 4, 10, 19, 8, 30, 4, 8, 38, 26, 19]
L_s = [2**i for i in L_s]
n_sym = 16
msg = [0,3,2,3,2,1,2,3,4,7,3,1,2,3,4,5,6,7, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [11]:
# test the encodeDecodePython function
res = encode_decode_python(L, s_list, L_s, n_sym, msg)
print(res)

: 

In [6]:
# time the function
import time

n_iter = 100

# Setting up the coder, then encoding and decoding (only tracks time of encoding and decoding)

coder_ptr = init_coder(L, s_list, L_s, n_sym)

# warm up
for i in range(10):
    result = encode_decode(coder_ptr, msg)

start = time.time()

for i in range(n_iter):
    result = encode_decode(coder_ptr, msg)
    
end = time.time()
print("Time no setup (microseconds):", (end - start) * 1e6 / n_iter)

# Setting up coder and encoding and decoding in one step (tracks time of all steps)
start = time.time()

for i in range(n_iter):
    result = encode_decode_python(L, s_list, L_s, n_sym, msg)
    
end = time.time()

print("Time w setup (microseconds):", (end - start) * 1e6 / n_iter)
result = encode_decode_python(L, s_list, L_s, n_sym, msg)
print("Compression ratio:", len(msg) * 4 / result)